<a href="https://colab.research.google.com/github/Bhagavanth67/HealthIntel-CLI/blob/main/6_HealthIntel_CLI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import files

filename = "healthintel_data.csv"

print("Welcome back to HealthIntel CLI!")
print("Today's focus: Error handling and debugging.\n")

# ----------------------------
# Safe input functions
# ----------------------------
def get_valid_name(prompt="Enter your name: "):
    """Ensures the name is not empty."""
    while True:
        name = input(prompt).strip()
        if name:
            return name
        print("Invalid input. Name cannot be empty.")

def get_valid_float(prompt, min_value=0):
    """Ensures input is a valid float and greater than min_value."""
    while True:
        try:
            value = float(input(prompt))
            if value > min_value:
                return value
            else:
                print(f"Please enter a number greater than {min_value}.")
        except ValueError:
            print("Invalid input. Please enter a numeric value.")

def get_valid_int(prompt, min_value=0):
    """Ensures input is a valid integer and greater than min_value."""
    while True:
        try:
            value = int(input(prompt))
            if value > min_value:
                return value
            else:
                print(f"Please enter a number greater than {min_value}.")
        except ValueError:
            print("Invalid input. Please enter an integer value.")

# ----------------------------
# Collect user input safely
# ----------------------------
try:
    name = get_valid_name()
    age = get_valid_int("Enter your age: ", 0)
    height = get_valid_float("Enter your height (in metres): ", 0.5)
    weight = get_valid_float("Enter your weight (in kilograms): ", 1.0)

    bmi = weight / (height ** 2)

    if bmi < 18.5:
        category = "Underweight"
    elif 18.5 <= bmi < 24.9:
        category = "Healthy weight"
    elif 25 <= bmi < 29.9:
        category = "Overweight"
    else:
        category = "Obese"

    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # ----------------------------
    # Create CSV file if missing
    # ----------------------------
    if not os.path.exists(filename):
        with open(filename, "w", newline="") as file:
            writer = csv.writer(file)
            writer.writerow(["DateTime", "Name", "Age", "Height (m)", "Weight (kg)", "BMI", "Category"])
        print("[DEBUG] CSV file created successfully.")

    # ----------------------------
    # Update existing record or add new
    # ----------------------------
    rows = []
    updated = False

    with open(filename, "r") as file:
        reader = csv.reader(file)
        header = next(reader)
        for row in reader:
            if row[1].strip().lower() == name.lower():
                print(f"[DEBUG] Updating existing record for {name}.")
                rows.append([timestamp, name, age, height, weight, round(bmi, 2), category])
                updated = True
            else:
                rows.append(row)

    if not updated:
        print(f"[DEBUG] Adding new record for {name}.")
        rows.append([timestamp, name, age, height, weight, round(bmi, 2), category])

    with open(filename, "w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(header)
        writer.writerows(rows)

    print("\nRecord saved successfully.")

    # ----------------------------
    # Load data and summarize
    # ----------------------------
    df = pd.read_csv(filename)
    print("\n=== All Recorded Users ===")
    print(df)

    total_users = len(df)
    average_bmi = round(df["BMI"].mean(), 2)
    category_counts = df["Category"].value_counts().to_dict()

    print("\n=== Health Statistics Summary ===")
    print(f"Total Users: {total_users}")
    print(f"Average BMI: {average_bmi}")
    print("Category Distribution:")
    for cat, count in category_counts.items():
        print(f"  {cat}: {count}")

    # ----------------------------
    # Visualize BMI Categories
    # ----------------------------
    plt.figure(figsize=(7, 5))
    df["Category"].value_counts().plot(kind="bar")
    plt.title("BMI Category Distribution")
    plt.xlabel("Category")
    plt.ylabel("Count")
    plt.grid(axis="y", linestyle="--", alpha=0.6)
    plt.tight_layout()
    plt.show()

    # ----------------------------
    # Visualize BMI Trend
    # ----------------------------
    df["DateTime"] = pd.to_datetime(df["DateTime"])
    df = df.sort_values(by="DateTime")

    plt.figure(figsize=(8, 5))
    plt.plot(df["DateTime"], df["BMI"], marker="o", linestyle="-")
    plt.title("BMI Trend Over Time")
    plt.xlabel("Date & Time")
    plt.ylabel("BMI Value")
    plt.xticks(rotation=45)
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()
    plt.show()

except Exception as e:
    print(f"\n[ERROR] Something went wrong: {e}")

finally:
    # Always try to download the CSV if it exists
    if os.path.exists(filename):
        print("\nDownloading latest health data CSV...")
        files.download(filename)
    else:
        print("\nNo CSV file found to download.")